# Infer-18-Decision-Value-Information : Valeur de l'Information

**Serie** : Programmation Probabiliste avec Infer.NET (18/20)  
**Duree estimee** : 45 minutes  
**Prerequis** : Notebooks 14-17 (Utilite et reseaux de decision)

---

## Objectifs

- Calculer la **valeur de l'information parfaite** (EVPI)
- Calculer la **valeur de l'information d'echantillon** (EVSI)
- Appliquer a des problemes classiques : droits de forage, chasse au tresor
- Comprendre quand l'information a de la valeur

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-17-Decision-Networks](Infer-17-Decision-Networks.ipynb) | [Infer-19-Decision-Expert-Systems](Infer-19-Decision-Expert-Systems.ipynb) |

---

## 1. Information et Reduction d'Incertitude

### Intuition

L'information a de la valeur quand elle **change notre decision**.

Si vous allez acheter un parapluie de toute facon, connaitre la meteo n'a pas de valeur.
Mais si vous hesitez, cette information peut etre precieuse.

### Question fondamentale

> Combien devrait-on payer pour obtenir une information avant de decider ?

### Types d'information

| Type | Description | Exemple |
|------|-------------|--------|
| **Parfaite** | Connaitre l'etat du monde avec certitude | Oracle parfait |
| **Imparfaite** | Signal bruite, test avec erreurs | Test medical |
| **Nulle** | Pas d'information | Statu quo |

In [1]:
// Installation Infer.NET
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;

Console.WriteLine("Infer.NET charge !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET charge !


## 2. Valeur de l'Information Parfaite (EVPI)

### Definition

$$\text{EVPI} = E[\max_a U(a|\omega)] - \max_a E[U(a)]$$

Ou :
- Le premier terme est l'utilite esperee **avec** information parfaite
- Le second terme est l'utilite esperee **sans** information

### Interpretation

> EVPI = Combien paierait un agent rationnel pour un oracle parfait ?

### Calcul

1. Sans information : choisir la meilleure action a priori
2. Avec information parfaite : pour chaque etat possible, choisir la meilleure action, puis faire la moyenne

In [2]:
// Calcul de l'EVPI : Exemple simple

// Scenario : Parapluie
// Etats : Pluie (P), Soleil (S)
// Actions : Prendre parapluie (U), Ne pas prendre (N)

double pPluie = 0.3;

// Matrice d'utilite U(action, etat)
var utilites = new Dictionary<(string action, string etat), double>
{
    { ("U", "P"), 70 },  // Parapluie + Pluie : protege mais encombrant
    { ("U", "S"), 60 },  // Parapluie + Soleil : encombrement inutile
    { ("N", "P"), 20 },  // Pas parapluie + Pluie : trempe !
    { ("N", "S"), 100 }, // Pas parapluie + Soleil : parfait
};

// Sans information : calculer E[U(a)] pour chaque action
double EU_U = pPluie * utilites[("U", "P")] + (1 - pPluie) * utilites[("U", "S")];
double EU_N = pPluie * utilites[("N", "P")] + (1 - pPluie) * utilites[("N", "S")];

double maxEU_sansInfo = Math.Max(EU_U, EU_N);
string bestAction_sansInfo = EU_U > EU_N ? "Prendre parapluie" : "Ne pas prendre";

Console.WriteLine("=== Sans Information ===");
Console.WriteLine($"P(pluie) = {pPluie:P0}\n");
Console.WriteLine($"E[U(prendre parapluie)] = {EU_U:F1}");
Console.WriteLine($"E[U(ne pas prendre)] = {EU_N:F1}");
Console.WriteLine($"=> Decision : {bestAction_sansInfo}");
Console.WriteLine($"=> E[U*] = {maxEU_sansInfo:F1}\n");

// Avec information parfaite
// Si pluie certaine : max(U(U,P), U(N,P)) = max(70, 20) = 70
// Si soleil certain : max(U(U,S), U(N,S)) = max(60, 100) = 100
double maxU_pluie = Math.Max(utilites[("U", "P")], utilites[("N", "P")]);
double maxU_soleil = Math.Max(utilites[("U", "S")], utilites[("N", "S")]);

double EU_avecInfo = pPluie * maxU_pluie + (1 - pPluie) * maxU_soleil;

Console.WriteLine("=== Avec Information Parfaite ===");
Console.WriteLine($"Si Pluie : action optimale donne U = {maxU_pluie}");
Console.WriteLine($"Si Soleil : action optimale donne U = {maxU_soleil}");
Console.WriteLine($"=> E[U* | info parfaite] = {EU_avecInfo:F1}\n");

double EVPI = EU_avecInfo - maxEU_sansInfo;
Console.WriteLine($"=== EVPI = {EVPI:F1} ===");
Console.WriteLine($"L'agent paierait jusqu'a {EVPI:F1} points pour connaitre la meteo a l'avance.");

=== Sans Information ===
P(pluie) = 30 %

E[U(prendre parapluie)] = 63,0
E[U(ne pas prendre)] = 76,0
=> Decision : Ne pas prendre
=> E[U*] = 76,0

=== Avec Information Parfaite ===
Si Pluie : action optimale donne U = 70
Si Soleil : action optimale donne U = 100
=> E[U* | info parfaite] = 91,0

=== EVPI = 15,0 ===
L'agent paierait jusqu'a 15,0 points pour connaitre la meteo a l'avance.


## 3. Exemple 1 : Droits de Forage Petrolier

### Scenario

Une compagnie petroliere possede des droits de forage sur un terrain.

- **Incertitude** : Presence de petrole (oui/non)
- **Decision** : Forer ou vendre les droits
- **Information possible** : Test sismique (imparfait)

### Donnees

| Parametre | Valeur |
|-----------|--------|
| P(petrole) | 30% |
| Cout forage | 500k |
| Revenu si petrole | 2M |
| Prix vente droits | 200k |
| Cout test sismique | 50k |
| Sensibilite test | 90% |
| Specificite test | 80% |

In [3]:
// Droits de forage petrolier

double pPetrole = 0.30;
double coutForage = 500_000;
double revenuPetrole = 2_000_000;
double prixVente = 200_000;
double coutTest = 50_000;
double sensibiliteTest = 0.90; // P(test+|petrole)
double specificiteTest = 0.80; // P(test-|pas petrole)

// Profits nets
double profit_forer_petrole = revenuPetrole - coutForage;  // 1.5M
double profit_forer_pasPetrole = -coutForage;              // -500k
double profit_vendre = prixVente;                          // 200k

Console.WriteLine("=== Droits de Forage Petrolier ===\n");

// 1. Sans information
double EU_forer = pPetrole * profit_forer_petrole + (1 - pPetrole) * profit_forer_pasPetrole;
double EU_vendre = profit_vendre;

double bestEU_sansInfo = Math.Max(EU_forer, EU_vendre);

Console.WriteLine("1. Sans information :");
Console.WriteLine($"   E[Profit(forer)] = {EU_forer:N0} EUR");
Console.WriteLine($"   E[Profit(vendre)] = {EU_vendre:N0} EUR");
Console.WriteLine($"   => Decision : {(EU_forer > EU_vendre ? "FORER" : "VENDRE")}\n");

// 2. EVPI
double maxProfit_petrole = Math.Max(profit_forer_petrole, profit_vendre);
double maxProfit_pasPetrole = Math.Max(profit_forer_pasPetrole, profit_vendre);
double EU_infoParf = pPetrole * maxProfit_petrole + (1 - pPetrole) * maxProfit_pasPetrole;

double EVPI = EU_infoParf - bestEU_sansInfo;

Console.WriteLine("2. Avec information parfaite :");
Console.WriteLine($"   Si petrole (30%) : forer => {maxProfit_petrole:N0}");
Console.WriteLine($"   Si pas petrole (70%) : vendre => {maxProfit_pasPetrole:N0}");
Console.WriteLine($"   E[Profit | info parfaite] = {EU_infoParf:N0} EUR");
Console.WriteLine($"   EVPI = {EVPI:N0} EUR\n");

=== Droits de Forage Petrolier ===

1. Sans information :
   E[Profit(forer)] = 100 000 EUR
   E[Profit(vendre)] = 200 000 EUR
   => Decision : VENDRE

2. Avec information parfaite :
   Si petrole (30%) : forer => 1 500 000
   Si pas petrole (70%) : vendre => 200 000
   E[Profit | info parfaite] = 590 000 EUR
   EVPI = 390 000 EUR



In [4]:
// 3. EVSI : Valeur de l'information du test sismique

// P(test+) = P(test+|petrole)P(petrole) + P(test+|pas petrole)P(pas petrole)
double pTestPos = sensibiliteTest * pPetrole + (1 - specificiteTest) * (1 - pPetrole);
double pTestNeg = 1 - pTestPos;

// P(petrole|test+) par Bayes
double pPetrole_testPos = (sensibiliteTest * pPetrole) / pTestPos;
// P(petrole|test-)
double pPetrole_testNeg = ((1 - sensibiliteTest) * pPetrole) / pTestNeg;

Console.WriteLine("3. Valeur du test sismique (EVSI) :");
Console.WriteLine($"   P(test+) = {pTestPos:P1}");
Console.WriteLine($"   P(petrole|test+) = {pPetrole_testPos:P1}");
Console.WriteLine($"   P(petrole|test-) = {pPetrole_testNeg:P1}\n");

// Decision si test positif
double EU_forer_testPos = pPetrole_testPos * profit_forer_petrole + (1 - pPetrole_testPos) * profit_forer_pasPetrole;
double bestEU_testPos = Math.Max(EU_forer_testPos, profit_vendre);
string decision_testPos = EU_forer_testPos > profit_vendre ? "FORER" : "VENDRE";

// Decision si test negatif
double EU_forer_testNeg = pPetrole_testNeg * profit_forer_petrole + (1 - pPetrole_testNeg) * profit_forer_pasPetrole;
double bestEU_testNeg = Math.Max(EU_forer_testNeg, profit_vendre);
string decision_testNeg = EU_forer_testNeg > profit_vendre ? "FORER" : "VENDRE";

Console.WriteLine($"   Si test+ : E[forer]={EU_forer_testPos:N0}, vendre={profit_vendre:N0} => {decision_testPos}");
Console.WriteLine($"   Si test- : E[forer]={EU_forer_testNeg:N0}, vendre={profit_vendre:N0} => {decision_testNeg}\n");

// E[Profit avec test] = P(test+) * best(test+) + P(test-) * best(test-) - cout_test
double EU_avecTest = pTestPos * bestEU_testPos + pTestNeg * bestEU_testNeg - coutTest;

double EVSI = (pTestPos * bestEU_testPos + pTestNeg * bestEU_testNeg) - bestEU_sansInfo;

Console.WriteLine($"   E[Profit avec test] = {EU_avecTest:N0} EUR");
Console.WriteLine($"   EVSI (avant cout) = {EVSI:N0} EUR");
Console.WriteLine($"   Cout test = {coutTest:N0} EUR");
Console.WriteLine($"   EVSI net = {EVSI - coutTest:N0} EUR\n");

// Decision finale
Console.WriteLine("=== Recommandation ===");
if (EU_avecTest > bestEU_sansInfo)
    Console.WriteLine($"Faire le test sismique (gain net = {EU_avecTest - bestEU_sansInfo:N0} EUR)");
else
    Console.WriteLine($"Ne pas faire le test, {(EU_forer > EU_vendre ? "forer directement" : "vendre directement")}");

Console.WriteLine($"\nEfficiency du test : EVSI/EVPI = {EVSI/EVPI:P0}");

3. Valeur du test sismique (EVSI) :
   P(test+) = 41,0 %
   P(petrole|test+) = 65,9 %
   P(petrole|test-) = 5,1 %

   Si test+ : E[forer]=817 073, vendre=200 000 => FORER
   Si test- : E[forer]=-398 305, vendre=200 000 => VENDRE

   E[Profit avec test] = 403 000 EUR
   EVSI (avant cout) = 253 000 EUR
   Cout test = 50 000 EUR
   EVSI net = 203 000 EUR

=== Recommandation ===
Faire le test sismique (gain net = 203 000 EUR)

Efficiency du test : EVSI/EVPI = 65 %


## 4. Exemple 2 : Chasse au Tresor

### Scenario

Un tresor est cache dans l'une des 5 cases d'une grille.
Chaque fouille coute 10 EUR. Le tresor vaut 100 EUR.

Un detecteur de metal peut indiquer (avec bruit) si le tresor est proche.

In [5]:
// Chasse au tresor avec Infer.NET

int nCases = 5;
double coutFouille = 10;
double valeurTresor = 100;

// Prior uniforme sur la position du tresor
double pTresorParCase = 1.0 / nCases;

Console.WriteLine("=== Chasse au Tresor ===\n");
Console.WriteLine($"Cases : {nCases}");
Console.WriteLine($"Cout par fouille : {coutFouille} EUR");
Console.WriteLine($"Valeur tresor : {valeurTresor} EUR");
Console.WriteLine($"Prior : P(tresor en case i) = {pTresorParCase:P0}\n");

// Strategie sans information : fouiller toutes les cases dans l'ordre
// E[cout] = somme sur k de P(tresor en case k) * (k * coutFouille)
// Avec prior uniforme et fouille sequentielle :
double EU_sansInfo = 0;
for (int k = 1; k <= nCases; k++)
{
    // Si tresor en case k, on fouille k cases
    EU_sansInfo += pTresorParCase * (valeurTresor - k * coutFouille);
}

Console.WriteLine("1. Sans information (fouille sequentielle) :");
Console.WriteLine($"   E[Profit] = {EU_sansInfo:F1} EUR\n");

// Avec information parfaite : on fouille directement la bonne case
double EU_infoParf = valeurTresor - coutFouille;
double EVPI_tresor = EU_infoParf - EU_sansInfo;

Console.WriteLine("2. Avec information parfaite :");
Console.WriteLine($"   E[Profit] = {EU_infoParf:F1} EUR");
Console.WriteLine($"   EVPI = {EVPI_tresor:F1} EUR");

=== Chasse au Tresor ===

Cases : 5
Cout par fouille : 10 EUR
Valeur tresor : 100 EUR
Prior : P(tresor en case i) = 20 %

1. Sans information (fouille sequentielle) :
   E[Profit] = 70,0 EUR

2. Avec information parfaite :
   E[Profit] = 90,0 EUR
   EVPI = 20,0 EUR


In [6]:
// Avec detecteur de metal (information imparfaite)

// Le detecteur indique "proche" ou "loin"
// P(proche|tresor dans case i) depend de la distance au centre
// Supposons detecteur pointe vers case 3 (centre)

int caseDetecteur = 3; // Case centrale
double pProcheGivenDistance(int distance) => Math.Exp(-distance); // Decroit avec distance

// Probabilites conditionnelles
Console.WriteLine("\n3. Detecteur de metal (centre en case 3) :\n");

double[] pProche = new double[nCases];
for (int i = 0; i < nCases; i++)
{
    int distance = Math.Abs(i + 1 - caseDetecteur);
    pProche[i] = pProcheGivenDistance(distance);
    Console.WriteLine($"   Case {i + 1}: distance={distance}, P(proche|tresor ici)={pProche[i]:F3}");
}

// P(detecteur dit proche)
double pSignalProche = pProche.Sum() / nCases;
Console.WriteLine($"\n   P(signal proche) = {pSignalProche:F3}");

// Posterieurs si signal proche (Bayes)
Console.WriteLine("\n   Si signal 'proche' :");
double[] posteriorProche = new double[nCases];
for (int i = 0; i < nCases; i++)
{
    posteriorProche[i] = (pProche[i] * pTresorParCase) / pSignalProche;
    Console.WriteLine($"     P(case {i + 1}|proche) = {posteriorProche[i]:F3}");
}

// Strategie optimale si proche : fouiller dans l'ordre des posterieurs
var ordreProche = Enumerable.Range(0, nCases)
    .OrderByDescending(i => posteriorProche[i])
    .ToArray();

Console.WriteLine($"\n   Ordre optimal si proche : {string.Join(" -> ", ordreProche.Select(i => $"Case {i+1}"))}");

// E[Profit | proche] avec strategie optimale
double EU_proche = 0;
double[] cumProb = new double[nCases];
for (int k = 0; k < nCases; k++)
{
    int caseIdx = ordreProche[k];
    EU_proche += posteriorProche[caseIdx] * (valeurTresor - (k + 1) * coutFouille);
}

Console.WriteLine($"   E[Profit | proche] = {EU_proche:F1} EUR");

// E[Profit | loin] (signal non-proche)
double[] posteriorLoin = new double[nCases];
for (int i = 0; i < nCases; i++)
{
    posteriorLoin[i] = ((1 - pProche[i]) * pTresorParCase) / (1 - pSignalProche);
}

var ordreLoin = Enumerable.Range(0, nCases)
    .OrderByDescending(i => posteriorLoin[i])
    .ToArray();

double EU_loin = 0;
for (int k = 0; k < nCases; k++)
{
    int caseIdx = ordreLoin[k];
    EU_loin += posteriorLoin[caseIdx] * (valeurTresor - (k + 1) * coutFouille);
}

Console.WriteLine($"   E[Profit | loin] = {EU_loin:F1} EUR");

// EVSI
double EU_avecDetecteur = pSignalProche * EU_proche + (1 - pSignalProche) * EU_loin;
double EVSI_detecteur = EU_avecDetecteur - EU_sansInfo;

Console.WriteLine($"\n   E[Profit avec detecteur] = {EU_avecDetecteur:F1} EUR");
Console.WriteLine($"   EVSI = {EVSI_detecteur:F1} EUR");
Console.WriteLine($"   Efficiency : {EVSI_detecteur / EVPI_tresor:P0} de l'info parfaite");


3. Detecteur de metal (centre en case 3) :

   Case 1: distance=2, P(proche|tresor ici)=0,135
   Case 2: distance=1, P(proche|tresor ici)=0,368
   Case 3: distance=0, P(proche|tresor ici)=1,000
   Case 4: distance=1, P(proche|tresor ici)=0,368
   Case 5: distance=2, P(proche|tresor ici)=0,135

   P(signal proche) = 0,401

   Si signal 'proche' :
     P(case 1|proche) = 0,067
     P(case 2|proche) = 0,183
     P(case 3|proche) = 0,498
     P(case 4|proche) = 0,183
     P(case 5|proche) = 0,067

   Ordre optimal si proche : Case 3 -> Case 2 -> Case 4 -> Case 1 -> Case 5
   E[Profit | proche] = 79,8 EUR
   E[Profit | loin] = 76,6 EUR

   E[Profit avec detecteur] = 77,8 EUR
   EVSI = 7,8 EUR
   Efficiency : 39 % de l'info parfaite


## 5. Quand l'Information a-t-elle de la Valeur ?

### Conditions pour EVPI > 0

L'information parfaite a de la valeur si et seulement si :

> La decision optimale **change** selon l'etat du monde.

Si la meme action est optimale quel que soit l'etat, EVPI = 0.

### Conditions pour EVSI > 0

L'information imparfaite a de la valeur si :

1. EVPI > 0 (l'info parfaite aurait de la valeur)
2. Le signal est **informatif** (change les posterieurs)
3. Les posterieurs causent des **changements de decision**

### Facteurs affectant la valeur

| Facteur | Effet sur VoI |
|---------|---------------|
| Incertitude plus grande | VoI augmente |
| Qualite du test (sens/spec) | VoI augmente |
| Cout de l'erreur | VoI augmente |
| Decision irreversible | VoI augmente |

In [7]:
// Analyse de sensibilite : EVPI en fonction de l'incertitude

Console.WriteLine("=== EVPI en Fonction de P(petrole) ===\n");
Console.WriteLine("P(petrole) | EU_forer  | EU_vendre | EVPI");
Console.WriteLine("-----------|-----------|-----------|--------");

for (double p = 0.1; p <= 0.9; p += 0.1)
{
    double eu_f = p * profit_forer_petrole + (1 - p) * profit_forer_pasPetrole;
    double eu_v = profit_vendre;
    double best_sans = Math.Max(eu_f, eu_v);
    
    double max_p = Math.Max(profit_forer_petrole, profit_vendre);
    double max_np = Math.Max(profit_forer_pasPetrole, profit_vendre);
    double eu_parfait = p * max_p + (1 - p) * max_np;
    
    double evpi = eu_parfait - best_sans;
    
    Console.WriteLine($"{p,10:P0} | {eu_f,9:N0} | {eu_v,9:N0} | {evpi,6:N0}");
}

Console.WriteLine();
Console.WriteLine("=> L'EVPI est maximal quand l'incertitude rend la decision difficile.");
Console.WriteLine("   Elle est nulle quand une action domine clairement.");

=== EVPI en Fonction de P(petrole) ===

P(petrole) | EU_forer  | EU_vendre | EVPI
-----------|-----------|-----------|--------
      10 % |  -300 000 |   200 000 | 130 000
      20 % |  -100 000 |   200 000 | 260 000
      30 % |   100 000 |   200 000 | 390 000
      40 % |   300 000 |   200 000 | 420 000
      50 % |   500 000 |   200 000 | 350 000
      60 % |   700 000 |   200 000 | 280 000
      70 % |   900 000 |   200 000 | 210 000
      80 % | 1 100 000 |   200 000 | 140 000
      90 % | 1 300 000 |   200 000 | 70 000

=> L'EVPI est maximal quand l'incertitude rend la decision difficile.
   Elle est nulle quand une action domine clairement.


## 6. Implementation Generique avec Infer.NET

In [8]:
// Calculateur generique de VoI avec Infer.NET

public class ValueOfInformation
{
    public string[] States { get; set; }
    public double[] Priors { get; set; }
    public string[] Actions { get; set; }
    public double[,] Utilities { get; set; } // Utilities[action, state]
    
    public (double euNoInfo, string bestActionNoInfo) ComputeWithoutInfo()
    {
        double maxEU = double.NegativeInfinity;
        string bestAction = null;
        
        for (int a = 0; a < Actions.Length; a++)
        {
            double eu = 0;
            for (int s = 0; s < States.Length; s++)
                eu += Priors[s] * Utilities[a, s];
            
            if (eu > maxEU)
            {
                maxEU = eu;
                bestAction = Actions[a];
            }
        }
        return (maxEU, bestAction);
    }
    
    public double ComputeEVPI()
    {
        var (euNoInfo, _) = ComputeWithoutInfo();
        
        double euPerfect = 0;
        for (int s = 0; s < States.Length; s++)
        {
            double maxUForState = double.NegativeInfinity;
            for (int a = 0; a < Actions.Length; a++)
                maxUForState = Math.Max(maxUForState, Utilities[a, s]);
            euPerfect += Priors[s] * maxUForState;
        }
        
        return euPerfect - euNoInfo;
    }
    
    public double ComputeEVSI(double[,] likelihood) // likelihood[signal, state]
    {
        int nSignals = likelihood.GetLength(0);
        var (euNoInfo, _) = ComputeWithoutInfo();
        
        double euWithSignal = 0;
        
        for (int sig = 0; sig < nSignals; sig++)
        {
            // P(signal)
            double pSignal = 0;
            for (int s = 0; s < States.Length; s++)
                pSignal += likelihood[sig, s] * Priors[s];
            
            // Posteriors P(state|signal)
            double[] posteriors = new double[States.Length];
            for (int s = 0; s < States.Length; s++)
                posteriors[s] = (likelihood[sig, s] * Priors[s]) / pSignal;
            
            // Best action given signal
            double maxEUGivenSignal = double.NegativeInfinity;
            for (int a = 0; a < Actions.Length; a++)
            {
                double eu = 0;
                for (int s = 0; s < States.Length; s++)
                    eu += posteriors[s] * Utilities[a, s];
                maxEUGivenSignal = Math.Max(maxEUGivenSignal, eu);
            }
            
            euWithSignal += pSignal * maxEUGivenSignal;
        }
        
        return euWithSignal - euNoInfo;
    }
}

// Test avec le probleme du forage
var voiCalculator = new ValueOfInformation
{
    States = new[] { "Petrole", "Pas Petrole" },
    Priors = new[] { 0.3, 0.7 },
    Actions = new[] { "Forer", "Vendre" },
    Utilities = new double[,]
    {
        { 1_500_000, -500_000 },  // Forer
        { 200_000, 200_000 }       // Vendre
    }
};

var (eu, best) = voiCalculator.ComputeWithoutInfo();
Console.WriteLine($"Sans info : {best} avec EU = {eu:N0}");
Console.WriteLine($"EVPI = {voiCalculator.ComputeEVPI():N0}");

// Likelihood du test sismique
var likelihood = new double[,]
{
    { 0.9, 0.2 },  // P(test+|state)
    { 0.1, 0.8 }   // P(test-|state)
};

Console.WriteLine($"EVSI (test sismique) = {voiCalculator.ComputeEVSI(likelihood):N0}");

Sans info : Vendre avec EU = 200 000
EVPI = 390 000
EVSI (test sismique) = 253 000


### 6.1 Visualisation Prior → Posterior avec Infer.NET

Cette section illustre **visuellement** comment l'observation d'un signal met a jour nos croyances et change potentiellement notre decision.

L'inference bayesienne d'Infer.NET calcule automatiquement les posterieurs, eliminant les calculs manuels de Bayes.

In [ ]:
// Demonstration Prior -> Posterior avec Infer.NET pour le forage petrolier

// Modele generatif avec Infer.NET
Variable<bool> petrole = Variable.Bernoulli(0.3).Named("petrole");
Variable<bool> testSismique = Variable.New<bool>().Named("test_sismique");

// P(test+|petrole) = 90% (sensibilite)
// P(test+|pas petrole) = 20% (1 - specificite)
using (Variable.If(petrole))
    testSismique.SetTo(Variable.Bernoulli(0.9));
using (Variable.IfNot(petrole))
    testSismique.SetTo(Variable.Bernoulli(0.2));

InferenceEngine engineVOI = new InferenceEngine();
engineVOI.Compiler.CompilerChoice = Microsoft.ML.Probabilistic.Compiler.CompilerChoice.Roslyn;

// Utilites
double U_forer_petrole = 1_500_000;
double U_forer_pasPetrole = -500_000;
double U_vendre = 200_000;

Console.WriteLine("=== Inference Bayesienne avec Infer.NET ===\n");

// 1. PRIOR (sans observation)
Console.WriteLine("1. PRIOR (avant le test sismique) :");
Bernoulli priorPetrole = engineVOI.Infer<Bernoulli>(petrole);
double p_prior = priorPetrole.GetProbTrue();
double EU_forer_prior = p_prior * U_forer_petrole + (1 - p_prior) * U_forer_pasPetrole;
Console.WriteLine($"   P(petrole) = {p_prior:P1}");
Console.WriteLine($"   E[U(forer)] = {EU_forer_prior:N0} EUR");
Console.WriteLine($"   E[U(vendre)] = {U_vendre:N0} EUR");
Console.WriteLine($"   => Decision : {(EU_forer_prior > U_vendre ? "FORER" : "VENDRE")}\n");

// 2. POSTERIOR si test POSITIF
testSismique.ObservedValue = true;
Console.WriteLine("2. POSTERIOR (test sismique POSITIF) :");
Bernoulli posteriorTestPos = engineVOI.Infer<Bernoulli>(petrole);
double p_pos = posteriorTestPos.GetProbTrue();
double EU_forer_pos = p_pos * U_forer_petrole + (1 - p_pos) * U_forer_pasPetrole;
Console.WriteLine($"   P(petrole|test+) = {p_pos:P1}  (etait {p_prior:P1})");
Console.WriteLine($"   E[U(forer)|test+] = {EU_forer_pos:N0} EUR");
Console.WriteLine($"   => Decision : {(EU_forer_pos > U_vendre ? "FORER" : "VENDRE")}");
Console.WriteLine($"   Impact : +{p_pos - p_prior:P1} de confiance\n");

// 3. POSTERIOR si test NEGATIF
testSismique.ClearObservedValue();
testSismique.ObservedValue = false;
Console.WriteLine("3. POSTERIOR (test sismique NEGATIF) :");
Bernoulli posteriorTestNeg = engineVOI.Infer<Bernoulli>(petrole);
double p_neg = posteriorTestNeg.GetProbTrue();
double EU_forer_neg = p_neg * U_forer_petrole + (1 - p_neg) * U_forer_pasPetrole;
Console.WriteLine($"   P(petrole|test-) = {p_neg:P1}  (etait {p_prior:P1})");
Console.WriteLine($"   E[U(forer)|test-] = {EU_forer_neg:N0} EUR");
Console.WriteLine($"   => Decision : {(EU_forer_neg > U_vendre ? "FORER" : "VENDRE")}");
Console.WriteLine($"   Impact : {p_neg - p_prior:P1} de confiance\n");

// Resume visuel
Console.WriteLine("=== Visualisation de l'Impact de l'Information ===");
Console.WriteLine();
Console.WriteLine($"Prior:          [{new string('#', (int)(p_prior * 50))}{new string('.', 50 - (int)(p_prior * 50))}] {p_prior:P0}");
Console.WriteLine($"Posterior T+:   [{new string('#', (int)(p_pos * 50))}{new string('.', 50 - (int)(p_pos * 50))}] {p_pos:P0}  -> FORER");
Console.WriteLine($"Posterior T-:   [{new string('#', (int)(p_neg * 50))}{new string('.', 50 - (int)(p_neg * 50))}] {p_neg:P0}  -> VENDRE");
Console.WriteLine();
Console.WriteLine("=> Le test change la decision : c'est pourquoi il a de la valeur !");

## 7. Exercice : Faut-il Faire un Test Medical ?

### Enonce

Un patient a 5% de chance d'avoir une maladie grave.
Un test coute 100 EUR, avec sensibilite 95% et specificite 90%.

Le traitement coute 2000 EUR et :
- Guerit 90% des malades (utilite 10000 si gueri)
- A des effets secondaires pour les non-malades (utilite 8000)

Sans traitement :
- Les malades ont utilite 3000 (maladie non traitee)
- Les non-malades ont utilite 10000 (parfait)

**Question** : Le test vaut-il son prix ?

In [9]:
// Exercice : Decision de test medical

// Parametres
double pMaladie_ex = 0.05;
double sensibilite_ex = 0.95;
double specificite_ex = 0.90;
double coutTest_ex = 100;
double coutTraitement = 2000;

// Utilites brutes
double U_traiter_malade_gueri = 10000 - coutTraitement;
double U_traiter_malade_echec = 3000 - coutTraitement; // Suppose meme utilite que non traite
double U_traiter_sain = 8000 - coutTraitement;
double U_pasTraiter_malade = 3000;
double U_pasTraiter_sain = 10000;

// Utilite esperee de traiter un malade (90% guerison)
double U_traiter_malade = 0.9 * U_traiter_malade_gueri + 0.1 * U_traiter_malade_echec;

Console.WriteLine("=== Exercice : Test Medical ===\n");
Console.WriteLine($"P(maladie) = {pMaladie_ex:P0}");
Console.WriteLine($"Cout test = {coutTest_ex} EUR\n");

// A vous de completer les calculs !
// 1. E[U] sans information pour chaque action
double EU_traiter = pMaladie_ex * U_traiter_malade + (1 - pMaladie_ex) * U_traiter_sain;
double EU_pasTraiter = pMaladie_ex * U_pasTraiter_malade + (1 - pMaladie_ex) * U_pasTraiter_sain;

Console.WriteLine("1. Sans test :");
Console.WriteLine($"   E[U(traiter)] = {EU_traiter:F0}");
Console.WriteLine($"   E[U(pas traiter)] = {EU_pasTraiter:F0}");
Console.WriteLine($"   => Decision : {(EU_traiter > EU_pasTraiter ? "TRAITER" : "PAS TRAITER")}\n");

// 2. EVPI
double maxU_malade = Math.Max(U_traiter_malade, U_pasTraiter_malade);
double maxU_sain = Math.Max(U_traiter_sain, U_pasTraiter_sain);
double EU_parfait = pMaladie_ex * maxU_malade + (1 - pMaladie_ex) * maxU_sain;
double EVPI_ex = EU_parfait - Math.Max(EU_traiter, EU_pasTraiter);

Console.WriteLine($"2. EVPI = {EVPI_ex:F0} EUR\n");

// 3. EVSI avec le test
double pTestPos_ex = sensibilite_ex * pMaladie_ex + (1 - specificite_ex) * (1 - pMaladie_ex);
double pM_TPos = (sensibilite_ex * pMaladie_ex) / pTestPos_ex;
double pM_TNeg = ((1 - sensibilite_ex) * pMaladie_ex) / (1 - pTestPos_ex);

double EU_traiter_TPos = pM_TPos * U_traiter_malade + (1 - pM_TPos) * U_traiter_sain;
double EU_pasTraiter_TPos = pM_TPos * U_pasTraiter_malade + (1 - pM_TPos) * U_pasTraiter_sain;
double bestEU_TPos = Math.Max(EU_traiter_TPos, EU_pasTraiter_TPos);

double EU_traiter_TNeg = pM_TNeg * U_traiter_malade + (1 - pM_TNeg) * U_traiter_sain;
double EU_pasTraiter_TNeg = pM_TNeg * U_pasTraiter_malade + (1 - pM_TNeg) * U_pasTraiter_sain;
double bestEU_TNeg = Math.Max(EU_traiter_TNeg, EU_pasTraiter_TNeg);

double EU_avecTest_ex = pTestPos_ex * bestEU_TPos + (1 - pTestPos_ex) * bestEU_TNeg;
double EVSI_ex = EU_avecTest_ex - Math.Max(EU_traiter, EU_pasTraiter);

Console.WriteLine($"3. EVSI = {EVSI_ex:F0} EUR");
Console.WriteLine($"   Cout test = {coutTest_ex} EUR");
Console.WriteLine($"   EVSI net = {EVSI_ex - coutTest_ex:F0} EUR\n");

Console.WriteLine($"=== Conclusion : {(EVSI_ex > coutTest_ex ? "FAIRE LE TEST" : "NE PAS FAIRE LE TEST")} ===");

=== Exercice : Test Medical ===

P(maladie) = 5 %
Cout test = 100 EUR

1. Sans test :
   E[U(traiter)] = 6065
   E[U(pas traiter)] = 9650
   => Decision : PAS TRAITER

2. EVPI = 215 EUR

3. EVSI = 0 EUR
   Cout test = 100 EUR
   EVSI net = -100 EUR

=== Conclusion : NE PAS FAIRE LE TEST ===


## 8. Resume

| Concept | Formule | Interpretation |
|---------|---------|----------------|
| **EVPI** | E[max U\|ω] - max E[U] | Valeur d'un oracle parfait |
| **EVSI** | E[max U\|signal] - max E[U] | Valeur d'un signal imparfait |
| **Efficiency** | EVSI / EVPI | Qualite relative de l'information |

### Conditions pour VoI > 0

1. L'information doit etre **pertinente** (changer les posterieurs)
2. Les posterieurs doivent **changer la decision**
3. Le cout de l'information doit etre inferieur a l'EVSI

---

## Pour aller plus loin

| Si vous voulez... | Consultez... |
|-------------------|-------------|
| Decisions robustes | [Infer-19-Decision-Expert-Systems](Infer-19-Decision-Expert-Systems.ipynb) |
| Decisions sequentielles | [Infer-20-Decision-Sequential](Infer-20-Decision-Sequential.ipynb) |

---

## References

- Raiffa & Schlaifer (1961) : Applied Statistical Decision Theory
- Russell & Norvig : AI, Chapter 16.6
- Howard (1966) : Information Value Theory